Querying/filtering of the Stats Can building permits data

full table downloaded from here:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3410028501

over 7gb or so unzipped

In [1]:
import pandas as pd

In [2]:
filename = "input-data/34100285.csv"

df = []

chunksize = 10 ** 6
with pd.read_csv(filename, chunksize=chunksize) as reader:
    for chunk in reader:
        filtered_chunk = chunk[chunk["Seasonal adjustment, value type"] == "Unadjusted, current"]
        filtered_chunk = filtered_chunk[
            filtered_chunk["Variables"] == "Number of dwelling-units created"
        ]
        filtered_chunk = filtered_chunk[
            filtered_chunk["Type of building"] == "Total residential and non-residential"
        ]
        filtered_chunk = filtered_chunk[
            (filtered_chunk["Type of work"] == "New construction") | 
            (filtered_chunk["Type of work"] == "Conversion from single to multiple dwelling") | 
            (filtered_chunk["Type of work"] == "Conversion from multiple to multiple dwelling") | 
            (filtered_chunk["Type of work"] == "Conversion from non-residential to residential building")
        ]
        if len(filtered_chunk) > 0:
            df.append(filtered_chunk)

In [3]:
df = pd.concat(df, ignore_index=True)

In [4]:
df['Type'] = df['Type of work'].replace(
    {
        'New construction': 'new',
        'Conversion from single to multiple dwelling': 's-to-m',
        'Conversion from multiple to multiple dwelling': 'm-to-m',
        'Conversion from non-residential to residential building': 'n-to-r'
    }
)

In [5]:
df = df[["REF_DATE","GEO", "Type","VALUE"]]

In [6]:
# df[df["GEO"] == "Toronto, Ontario"].to_csv("input-data/toronto-subset.csv")

In [7]:
df.to_csv("conversions-canada-2018-to-2024.csv", index=False)

In [8]:
df

,REF_DATE,GEO,Type,VALUE
0,2018-01,Canada,new,13498.0
1,2018-01,Canada,s-to-m,475.0
2,2018-01,Canada,m-to-m,384.0
3,2018-01,Canada,n-to-r,143.0
4,2018-01,Newfoundland and Labrador,new,23.0
...,...,...,...,...
20059,2024-04,"Vancouver, British Columbia",n-to-r,4.0
20060,2024-04,"Victoria, British Columbia",new,727.0
20061,2024-04,"Victoria, British Columbia",s-to-m,16.0
20062,2024-04,"Victoria, British Columbia",m-to-m,0.0
